In [1]:
# change to `%matplotlib qt5` for interactive methods
%matplotlib qt5
from pathlib import Path
from imgseries import ImgSeries, ContourTracking, ContourTrackingResults

# Define image series to work on

In [2]:
basefolder = Path('data')
folders = [basefolder / folder for folder in ('img1', 'img2')]
images = ImgSeries(folders, savepath=basefolder)

# Minimal analysis

In [3]:
ct = ContourTracking(images)

The line below assumes that contours to follow have already been defined and saved in the metadata file (see details further below)

In [4]:
ct.regenerate()

In [18]:
ct.contours.show()

<Axes: title={'center': 'img #10, grey level 181'}>

Now, run analysis on these zones.

In [19]:
ct.run()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 153.42it/s]


Results are stored in the `results.data` attribute, which is a pandas DataFrame (times are automatically extracted from image creation date, but can be modified, see further below)). `x, y` represent position, `p` perimeter and `a` signed area (see **imgbasics** package)

*Note*: if contour detection fails at some point, `data` will contain `NaN` (`numpy.nan`) at the corresponding locations.

*Note*: by default, the full coordinates of the contours are also stored in `results.raw_contour_data`. If this is too heavey, `ContourTracking` can be instanciated with the option `save_raw_contours=False`

In [20]:
ct.results.data.head()

,folder,filename,time (unix),x1,y1,p1,a1,x2,y2,p2,a2,x3,y3,p3,a3
num,,,,,,,,,,,,,,,
0,img1,img-00610.png,1.696408e+09,153.206899,249.426678,42.934109,-142.882202,219.912664,278.198407,51.194526,-204.722955,292.745765,270.21115,46.271683,-169.317036
1,img1,img-00611.png,1.696408e+09,153.389457,249.22774,42.778324,-141.810327,219.9242,277.918486,51.108036,-203.94459,292.570581,269.946013,46.066067,-167.753201
2,img1,img-00612.png,1.696408e+09,153.532051,249.139501,42.590909,-140.473727,219.871375,277.773556,50.95654,-202.729721,292.342911,269.789197,45.919486,-166.637596
3,img1,img-00613.png,1.696408e+09,153.6559,248.967076,42.464292,-139.578893,219.799774,277.51511,50.920435,-202.361735,292.103532,269.530549,45.718974,-165.141563
4,img1,img-00614.png,1.696408e+09,153.766926,248.799268,42.218459,-137.956127,219.746896,277.284304,50.848014,-202.013301,291.876028,269.301173,45.611392,-164.326956


In [21]:
ct.results.raw_contour_data['1']['10']['y'][:10]

[255.30434782608697,
 255.75,
 255.95833333333334,
 255.95454545454547,
 255.71428571428572,
 255.26315789473685,
 255.0,
 254.61111111111111,
 254.0,
 253.7058823529412]

In [22]:
ct.inspect(skip=3)

In [23]:
ct.animate(start=10, end=30)

Plot perimeters of all detected particles (contours) as a function of time:

In [24]:
ct.results.data.set_index('time (unix)').filter(like='p').plot()

<Axes: xlabel='time (unix)'>

Save data in a tsv (tab-separated value) file, using default filename (filename can be set as a parameter if necessary, see further below). Metadata including contour info, path info, and code version info is also saved in a .json file at the same time. Full (raw) contour data is also saved if required.

Before saving, make sure the timing info is correct for all images. If not, correct timing info, re-run the analysis, and call `results.save()`.

In [25]:
ct.results.save()

# Live view of analysis

In [26]:
ct.run(end=30, live=True)

It is possible that the live animation is non blocking and thus that data is not saved correctly (saving is done at the beginning of the animation and thus data is empty). If this happens, best is probably to re-run the analysis with live=False.

Check that data is empty:

In [28]:
ct.results.data.set_index('time (unix)').filter(like='p').plot()

<Axes: xlabel='time (unix)'>

# Defining and viewing contours

Defining contours has to be done at least once.

**Important**: Matplotlib must be in an interactive mode to do so.

Defining does not need to be done again in the following situations:
- calling methods again from the same `ct` object, e.g. `ct.run()`
- calling `ct.contours.load()` or `ct.regenerate()` to load contours data from saved metadata (.json) file.

In [9]:
images.crop.define()

In [10]:
images.filter.define()  # optional, smoothing of image
# (or images.filter.type = 'gaussian', images.filter.size = 2.2)

In [11]:
ct.threshold.define()

In [12]:
print(images.filter)
print(ct.threshold)

Filter object {'type': 'gaussian', 'size': 2.0}
Threshold object {'value': 181}


In [13]:
ct.contours.define()  # define one contour on the first image of the series

In [14]:
ct.contours.define(n=3)  # define 3 contours on the first image of the series

In [15]:
ct.contours.define(3, num=10)  # define 3 contours at level 170 on image #10 in the series

Viewing analysis zones after defining or loading them:

In [16]:
ct.contours.data

{'position': {'contour 1': (155.34855196067068, 248.8680542590701),
  'contour 2': (220.36855953462214, 277.00651519592867),
  'contour 3': (291.47083356083385, 268.9632883107061)},
 'level': 181,
 'image': 10}

In [17]:
ct.contours.show()  # show contours on the image they have been defined on

<Axes: title={'center': 'img #10, grey level 181'}>

**Note**: At the moment, saving contours selection data is done by calling `ct.results.save()`, which saves both data and metadata. Be careful because calling `ct.results.save()` also overwrites saved analysis data (potentially with no data if no analysis has been run yet). This makes sure that metadata in .json files actually corresponds to the data in the .tsv file.

# Load analysis data a posteriori

## Load results without images

In [29]:
results = ContourTrackingResults(savepath=basefolder)

In [30]:
results.load()
results.data.head()

,folder,filename,time (unix),x1,y1,p1,a1,x2,y2,p2,a2,x3,y3,p3,a3
num,,,,,,,,,,,,,,,
0,img1,img-00610.png,1.696408e+09,153.206899,249.426678,42.934109,-142.882202,219.912664,278.198407,51.194526,-204.722955,292.745765,270.211150,46.271683,-169.317036
1,img1,img-00611.png,1.696408e+09,153.389457,249.227740,42.778324,-141.810327,219.924200,277.918486,51.108036,-203.944590,292.570581,269.946013,46.066067,-167.753201
2,img1,img-00612.png,1.696408e+09,153.532051,249.139501,42.590909,-140.473727,219.871375,277.773556,50.956540,-202.729721,292.342911,269.789197,45.919486,-166.637596
3,img1,img-00613.png,1.696408e+09,153.655900,248.967076,42.464292,-139.578893,219.799774,277.515110,50.920435,-202.361735,292.103532,269.530549,45.718974,-165.141563
4,img1,img-00614.png,1.696408e+09,153.766926,248.799268,42.218459,-137.956127,219.746896,277.284304,50.848014,-202.013301,291.876028,269.301173,45.611392,-164.326956


In [31]:
results.metadata

{'path': ['/Users/olivier.vincent/Python-OV/imgseries/data'],
 'folders': ['img1', 'img2'],
 'contours': {'position': {'contour 1': [155.34855196067068,
    248.8680542590701],
   'contour 2': [220.36855953462214, 277.00651519592867],
   'contour 3': [291.47083356083385, 268.9632883107061]},
  'level': 181,
  'image': 10},
 'grayscale': {},
 'rotation': {},
 'crop': {'zone': [86, 73, 454, 391]},
 'filter': {'type': 'gaussian', 'size': 2.0},
 'subtraction': {},
 'threshold': {},
 'time (utc)': '2024-01-28 19:45:29',
 'code version': {'skimage': {'status': 'not a git repository',
   'tag': 'v0.19.3'},
  'imgseries': {'hash': 'd4a58695989a69999f36e51c4ec4245ea362041b',
   'status': 'dirty'},
  'imgbasics': {'status': 'not a git repository', 'tag': 'v0.3.0'},
  'filo': {'hash': '150574b5ae82c74d0c500b3fdb494e8f9c7631f7',
   'status': 'clean',
   'tag': 'v1.1.5'},
  'matplotlib': {'status': 'not a git repository', 'tag': 'v3.7.0'},
  'numpy': {'status': 'not a git repository', 'tag': 'v1.23

In [32]:
results.raw_contour_data['2']['33']['x'][:10]

[223.0,
 222.0,
 221.0,
 220.0,
 219.0,
 218.5,
 218.0,
 217.0,
 216.0,
 215.93333333333334]

## Connect results to image series (e.g. for inspection/visualization)

In [33]:
images = ImgSeries(folders, savepath=basefolder)
ct = ContourTracking(images)
ct.regenerate()

In [34]:
print('Image Crop -------------------------', ct.img_series.crop.zone)
print('Image Crop (same as above) ---------', ct.results.metadata['crop'].get('zone', ()))
print('')
print('Image Rotation ---------------------', ct.img_series.rotation.data)
print('Image Rotation (same as above) -----', ct.results.metadata['rotation'])
print('')
print('Contour position -------------------', ct.contours.data['position']['contour 2'])
print('Contour position (same as above) ---', ct.results.metadata['contours']['position']['contour 2'])

ct.results.data.head()

Image Crop ------------------------- [86, 73, 454, 391]
Image Crop (same as above) --------- [86, 73, 454, 391]

Image Rotation --------------------- {}
Image Rotation (same as above) ----- {}

Contour position ------------------- [220.36855953462214, 277.00651519592867]
Contour position (same as above) --- [220.36855953462214, 277.00651519592867]


,folder,filename,time (unix),x1,y1,p1,a1,x2,y2,p2,a2,x3,y3,p3,a3
num,,,,,,,,,,,,,,,
0,img1,img-00610.png,1.696408e+09,153.206899,249.426678,42.934109,-142.882202,219.912664,278.198407,51.194526,-204.722955,292.745765,270.211150,46.271683,-169.317036
1,img1,img-00611.png,1.696408e+09,153.389457,249.227740,42.778324,-141.810327,219.924200,277.918486,51.108036,-203.944590,292.570581,269.946013,46.066067,-167.753201
2,img1,img-00612.png,1.696408e+09,153.532051,249.139501,42.590909,-140.473727,219.871375,277.773556,50.956540,-202.729721,292.342911,269.789197,45.919486,-166.637596
3,img1,img-00613.png,1.696408e+09,153.655900,248.967076,42.464292,-139.578893,219.799774,277.515110,50.920435,-202.361735,292.103532,269.530549,45.718974,-165.141563
4,img1,img-00614.png,1.696408e+09,153.766926,248.799268,42.218459,-137.956127,219.746896,277.284304,50.848014,-202.013301,291.876028,269.301173,45.611392,-164.326956


Once the analysis is regenerated, all the tools associated with images (inspection, showing, animation, etc.) are available:

In [35]:
ct.show(num=25)

(<Axes: title={'center': 'img #25, grey level None'}>,)

In [36]:
ct.inspect()

# Analyze only subset of images

See **Examples_GreyLevels.ipynb**, and replace:
- `GreyLevel` →  `ContourTracking`
- `zones` →  `contours`